<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/small_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import copy

In [2]:
# Load the training data (CIFAR10 to start)
training_data = datasets.CIFAR10(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

# Load the test data
test_data = datasets.CIFAR10(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
# Check the data dimensions
for X, y in test_dataloader:
  print("Shape of X batch [Batch size, Channels, Height, Width]: ", X.shape)
  print("Shape of y batch: ", y.shape, y.dtype)
  break

Shape of X batch [Batch size, Channels, Height, Width]:  torch.Size([64, 3, 32, 32])
Shape of y batch:  torch.Size([64]) torch.int64


# Building basic convolutional neural nets

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [37]:
# Define the model
class basicConvNet(nn.Module):
  def __init__(self, name):
    super(basicConvNet, self).__init__()
    self.name = name
    self.basic_conv_net = nn.Sequential(
        # 1st layer
        nn.Conv2d(3, 128, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d((2,2)),
        # 2nd layer
        nn.Conv2d(128, 64, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d((2,2)),
        # 3rd layer
        nn.Conv2d(64, 64, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d((2,2)),
        # 4th layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.ReLU(),
        #nn.MaxPool2d((2,2)),
        # 5th layer
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.ReLU(),
        #nn.MaxPool2d((2,2)),

        # Output layer
        nn.Flatten(),
        nn.Linear(32 * 4 * 4, 10),
    )
    self.softmax = nn.Softmax(dim=1)

  # Forward propagation
  def forward(self, x):
    x = self.basic_conv_net(x)
    x = self.softmax(x)
    return x
      

In [38]:
model = basicConvNet('test').to(device)

In [39]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [40]:
# Training loop
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [41]:
# Test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [42]:
n_epochs = 100

In [43]:
def training_loop(n_epochs, model, train_data, test_data, loss_function,
                  optimizer, early_stopping=False, patience=10):
  current_epoch = 0
  best_epoch = 0
  best_loss = float('inf')
  patience_counter = 0

  for e in range(n_epochs):
    print(f"Epoch {e+1}\n-----------------------------")
    train(train_data, model, loss_function, optimizer)
    test_loss = test(test_data, model, loss_function)

    # Iterate epoch counter
    current_epoch += 1

    # If early_stopping check test_loss
    if early_stopping:
      # case: test loss beats the current best loss
      if test_loss < best_loss:
        # store loss
        best_loss = test_loss

        # reset patience counter
        patience_counter = 0

        # store model and epoch number
        print("Storing new best model.")
        best_model_state_dict = copy.deepcopy(model.state_dict)
        best_epoch = current_epoch

      # Case: patience limit not yet reached => iterate patience counter
      elif patience_counter < patience - 1:
        patience_counter += 1
        print(f"Patience count: {patience_counter}")

      # Case: patience limit reached
      else:
        print("Finished due to early stopping.")
        print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
        torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')
        break

  # If we get here, we did not stop early - save best model
  if early_stopping:
    print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
    torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')

In [44]:
training_loop(n_epochs, model, train_dataloader, test_dataloader, loss_fn,
              optimizer, early_stopping=True)

Epoch 1
-----------------------------
loss: 2.302786 [    0/50000]
loss: 2.302409 [ 6400/50000]
loss: 2.302666 [12800/50000]
loss: 2.303092 [19200/50000]
loss: 2.302960 [25600/50000]
loss: 2.303239 [32000/50000]
loss: 2.302238 [38400/50000]
loss: 2.302965 [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302580 

Storing new best model.
Epoch 2
-----------------------------
loss: 2.302776 [    0/50000]
loss: 2.302406 [ 6400/50000]
loss: 2.302648 [12800/50000]
loss: 2.303088 [19200/50000]
loss: 2.302943 [25600/50000]
loss: 2.303223 [32000/50000]
loss: 2.302238 [38400/50000]
loss: 2.302948 [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302571 

Storing new best model.
Epoch 3
-----------------------------
loss: 2.302768 [    0/50000]
loss: 2.302405 [ 6400/50000]
loss: 2.302630 [12800/50000]
loss: 2.303083 [19200/50000]
loss: 2.302926 [25600/50000]
loss: 2.303209 [32000/50000]
loss: 2.302238 [38400/50000]
loss: 2.302932 [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg los